In [2]:
import torch
from torch.utils.data import DataLoader
from transformers import (AutoTokenizer,AutoModelForSequenceClassification,get_scheduler)
from torch.optim import AdamW
from datasets import load_dataset
from sklearn.metrics import accuracy_score
import numpy as np
from dataclasses import dataclass

In [21]:
#Config Class
@dataclass
class Config:
    model_name: str = "bert-base-uncased"
    batch_size: int = 16
    learning_rate: float = 2e-5
    num_epochs: int = 8
    max_length: int = 256
    weight_decay: float = 0.05 #initial was 0.01
    save_path: str = "./best_model.pt"
    device: str = "cuda" if torch.cuda.is_available() else "cpu"

config = Config()

In [3]:
!datasets-cli delete-cache stanfordnlp/imdb

usage: datasets-cli <command> [<args>]
HuggingFace Datasets CLI tool: error: argument {convert,env,test,run_beam,dummy_data}: invalid choice: 'delete-cache' (choose from 'convert', 'env', 'test', 'run_beam', 'dummy_data')


In [4]:
!pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [4]:
#Load IMDb Dataset
dataset = load_dataset("stanfordnlp/imdb")

#Tokenization
tokenizer = AutoTokenizer.from_pretrained(config.model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=config.max_length)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

train_dataset = tokenized_datasets["train"]
test_dataset = tokenized_datasets["test"]


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
#Check dataset
print(dataset["train"].select(range(5)))


Dataset({
    features: ['text', 'label'],
    num_rows: 5
})


In [23]:
from torch.cuda.amp import autocast, GradScaler
#Set Dataloaders
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=config.batch_size,num_workers=2,pin_memory=True)
eval_dataloader = DataLoader(test_dataset, batch_size=config.batch_size,num_workers=2,pin_memory=True)

#Load Model
from torch.nn import Dropout

model = AutoModelForSequenceClassification.from_pretrained(config.model_name, num_labels=2)

# Add dropout layers - 0.2 gives the best output for eval loss
model.dropout = Dropout(0.2) #initial was 0.1
model.classifier.dropout = Dropout(0.2) #initial was 0.1

model.to(config.device)

# Set Optimizer and Scheduler
optimizer = AdamW(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=len(train_dataloader) * config.num_epochs
)
scaler = GradScaler() # Initialize GradScaler

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-23-869aff84b6e9>:24: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler() # Initialize GradScaler


In [24]:
#Training Loop
from torch.cuda.amp import autocast, GradScaler
patience = 2
epochs_no_improve = 0
best_eval_loss = float("inf")
scaler = GradScaler() # Initialize GradScaler

for epoch in range(config.num_epochs):
    model.train()
    total_train_loss = 0

    for batch in train_dataloader:
        batch = {k: v.to(config.device) for k, v in batch.items()}

        with autocast(): # Enable autocast
            outputs = model(**batch)
            loss = outputs.loss

        total_train_loss += loss.item()
        #omiited loss and added scaler to improve time
        scaler.scale(loss).backward() # Scale the loss and backpropagate
        scaler.step(optimizer) # Update optimizer
        scaler.update() # Update the scaler
        lr_scheduler.step()
        optimizer.zero_grad()

    print(f"Epoch {epoch+1} | Train Loss: {total_train_loss / len(train_dataloader):.4f}")

    # Evaluation
    model.eval()
    total_eval_loss = 0
    preds, labels = [], []

    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(config.device) for k, v in batch.items()}

            with autocast(): # Enable autocast
              outputs = model(**batch)
              loss = outputs.loss
              logits = outputs.logits

            total_eval_loss += loss.item()
            preds.extend(torch.argmax(logits, dim=-1).cpu().numpy())
            labels.extend(batch["labels"].cpu().numpy())

    eval_loss = total_eval_loss / len(eval_dataloader)
    accuracy = accuracy_score(labels, preds)
    print(f"Epoch {epoch+1} | Eval Loss: {eval_loss:.4f} | Accuracy: {accuracy:.4f}")

    # Save Best Model and Check for Early Stopping
    if eval_loss < best_eval_loss:
       best_eval_loss = eval_loss
       torch.save(model.state_dict(), config.save_path)
       epochs_no_improve = 0
       print("Saved best model.")
    else:
       epochs_no_improve += 1
       print(f"No improvement for {epochs_no_improve} epochs.")

    # Early Stopping Check
    if epochs_no_improve >= patience:
       print(f"Early stopping triggered after {epoch+1} epochs.")
       break

<ipython-input-24-68b2788ec478>:6: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler() # Initialize GradScaler
<ipython-input-24-68b2788ec478>:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(): # Enable autocast


Epoch 1 | Train Loss: 0.2674


<ipython-input-24-68b2788ec478>:38: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(): # Enable autocast


Epoch 1 | Eval Loss: 0.2046 | Accuracy: 0.9200
Saved best model.


<ipython-input-24-68b2788ec478>:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(): # Enable autocast


...

Epoch 2 | Eval Loss: 0.2145 | Accuracy: 0.9226
No improvement for 1 epochs.


<ipython-input-24-68b2788ec478>:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(): # Enable autocast


Epoch 3 | Train Loss: 0.0619


<ipython-input-24-68b2788ec478>:38: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(): # Enable autocast


Epoch 3 | Eval Loss: 0.2582 | Accuracy: 0.9223
No improvement for 2 epochs.
Early stopping triggered after 3 epochs.


In [ ]:
#Training Loop without early stopping
from torch.cuda.amp import autocast, GradScaler
#patience = 2
#epochs_no_improve = 0
best_eval_loss = float("inf")
scaler = GradScaler() # Initialize GradScaler

for epoch in range(config.num_epochs):
    model.train()
    total_train_loss = 0

    for batch in train_dataloader:
        batch = {k: v.to(config.device) for k, v in batch.items()}

        with autocast(): # Enable autocast
            outputs = model(**batch)
            loss = outputs.loss

        total_train_loss += loss.item()
        #omiited loss and added scaler to improve time
        scaler.scale(loss).backward() # Scale the loss and backpropagate
        scaler.step(optimizer) # Update optimizer
        scaler.update() # Update the scaler
        lr_scheduler.step()
        optimizer.zero_grad()

    print(f"Epoch {epoch+1} | Train Loss: {total_train_loss / len(train_dataloader):.4f}")

    # Evaluation
    model.eval()
    total_eval_loss = 0
    preds, labels = [], []

    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(config.device) for k, v in batch.items()}

            with autocast(): # Enable autocast
              outputs = model(**batch)
              loss = outputs.loss
              logits = outputs.logits

            total_eval_loss += loss.item()
            preds.extend(torch.argmax(logits, dim=-1).cpu().numpy())
            labels.extend(batch["labels"].cpu().numpy())

    eval_loss = total_eval_loss / len(eval_dataloader)
    accuracy = accuracy_score(labels, preds)
    print(f"Epoch {epoch+1} | Eval Loss: {eval_loss:.4f} | Accuracy: {accuracy:.4f}")

    # Save Best Model and Check for Early Stopping
    if eval_loss < best_eval_loss:
       best_eval_loss = eval_loss
       torch.save(model.state_dict(), config.save_path)
       epochs_no_improve = 0
       print("Saved best model.")
    else:
       epochs_no_improve += 1
       print(f"No improvement for {epochs_no_improve} epochs.")

    # Early Stopping Check
    #if epochs_no_improve >= patience:
       #print(f"Early stopping triggered after {epoch+1} epochs.")
       #break # Exit the training loop

<ipython-input-25-5bfbf294e787>:6: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler() # Initialize GradScaler
<ipython-input-25-5bfbf294e787>:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(): # Enable autocast


Epoch 1 | Train Loss: 0.0304


<ipython-input-25-5bfbf294e787>:38: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(): # Enable autocast


Epoch 1 | Eval Loss: 0.2992 | Accuracy: 0.9221
Saved best model.


<ipython-input-25-5bfbf294e787>:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(): # Enable autocast


...

<ipython-input-25-5bfbf294e787>:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(): # Enable autocast


Epoch 6 | Train Loss: 0.0031


<ipython-input-25-5bfbf294e787>:38: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(): # Enable autocast


Epoch 6 | Eval Loss: 0.4210 | Accuracy: 0.9240
No improvement for 5 epochs.


<ipython-input-25-5bfbf294e787>:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(): # Enable autocast


In [14]:
#Load Best Model
model.load_state_dict(torch.load(config.save_path))
model.eval()

# Use model for prediction
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=config.max_length).to(config.device)
    with torch.no_grad():
        logits = model(**inputs).logits
    prediction = torch.argmax(logits, dim=-1).item()
    return "positive" if prediction == 1 else "negative"

# Example
print(predict("This movie was absolutely wonderful!"))

positive


In [ ]:
#More examples
new_texts = [
    "This is a fantastic film, I loved every minute of it!",
    "This movie was a complete waste of time.",
    "It was okay, nothing special.",
    "The acting was superb, but the story was weak.",
    "The acting was weak, but the story was superb.",
    "One of the best movies I've seen this year!"
]

print("\nPredictions on new data:")
for text in new_texts:
    sentiment = predict(text)
    print(f"'{text}' -> {sentiment}")
